In [1]:
#ResNet

In [2]:
#ResNet은 깊은 신경망을 학습하기 위해 개발된 모델
#잔차 학습 개념을 도입하여 매우 깊은 네트워크에서도 효율적인 학습이 가능하도록 함.
#딥러닝 모델이 너무 깊어질 때 발생하는 기울기 소실 문제를 해결

In [3]:
#ResNet의 기본 개념
#1. 깊은 신경망의 문제
# - 깊은 신경망은 더 많은 계층을 쌓아 복잡한 패턴을 학습할 수 있지만, 너무 깊어지면 학습이 어려워지는 문제.
# - 주로 기울기 소실이나 기울기 폭발같은 현상 떄문에 발생
# - 이는 모델이 더 이상 깊어지지 못하고 성능이 저하되는 결과를 초래

#2. 잔차 학습
# - ResNet은 이런 문제를 해결하기 위해 잔차 학습을 도입
# - 잔차 학습은 각 층의 출력이 바로 다음 층의 입력으로 전달되지 않고, 이전 층의 입력을 더해줌으로써 학습을 도움.
# - 이를 통해 기울기 소실 문제를 완화할 수 있음.

In [4]:
#ResNet의 주요 특징

# 기울기 소실 문제 해결
# - 잔차 학습을 통해 깊은 네트워크에서도 기울기 소실 문제를 해결
# - 입력을 출력에 더해줌으로써 신호가 더욱 쉽게 전달되어 학습이 원활하게 이루어짐.

# 간단한 블록 구조
# - ResNet은 간단한 블록 구조를 사용하여 네트워크를 쉽게 확장할 수 있음.

# 높은 성능
# - ResNet은 이미지 분류, 객체 검출 등 다양한 컴퓨터 비전 작업에서 높은 성능을 발휘
# - 깊은 네트워크에서도 안정적으로 학습할 수 있어, 복잡한 패턴을 잘 학습함.

In [5]:
#ResNet 실습
import torch
import torch.nn as nn
import torch.nn.functional as F

class Block(nn.Module):
    def __init__(self, in_ch, out_ch, stride=1):
        super(Block, self).__init__()
        # 첫 번째 컨볼루션 레이어
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_ch)  # 배치 정규화
        # 두 번째 컨볼루션 레이어
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_ch)  # 배치 정규화

        # 입력과 출력의 차원이 다를 경우 shortcut 경로 정의
        self.skip_connection = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.skip_connection = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),  # 차원 맞추기 위한 1x1 컨볼루션
                nn.BatchNorm2d(out_ch)  # 배치 정규화
            )
        
    def forward(self, x):
        # 첫 번째 컨볼루션 + ReLU 활성화 함수
        output = F.relu(self.bn1(self.conv1(x)))
        # 두 번째 컨볼루션 후 배치 정규화
        output = self.bn2(self.conv2(output))
        # shortcut 경로 출력과 현재 블록의 출력 더하기
        output += self.skip_connection(x)
        # 최종 ReLU 활성화 함수 적용
        output = F.relu(output)
        return output

# ResNet 모델 정의
class CustomResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(CustomResNet, self).__init__()
        self.initial_channels = 64  # 첫 번째 레이어의 입력 채널 수 정의
        # 첫 번째 컨볼루션 레이어
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)  # 배치 정규화
        # ResNet의 각 레이어 생성
        self.layer1 = self._create_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._create_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._create_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._create_layer(block, 512, layers[3], stride=2)
        # 평균 풀링 레이어
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        # 최종 완전 연결 레이어
        self.fc = nn.Linear(512, num_classes)
        
    # ResNet의 각 레이어를 생성하는 함수
    def _create_layer(self, block, out_ch, num_layers, stride):
        layer_list = []
        # 첫 번째 블록은 stride를 받을 수 있음
        layer_list.append(block(self.initial_channels, out_ch, stride))
        self.initial_channels = out_ch  # 다음 블록을 위해 채널 수 업데이트
        # 나머지 블록들은 기본 stride를 사용
        for _ in range(1, num_layers):
            layer_list.append(block(out_ch, out_ch))
        return nn.Sequential(*layer_list)
    
    def forward(self, x):
        # 첫 번째 컨볼루션 + ReLU 활성화 함수
        x = F.relu(self.bn1(self.conv1(x)))
        # 각 레이어를 순차적으로 통과
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # 평균 풀링 및 텐서의 차원 축소
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        # 최종 완전 연결 레이어를 통해 클래스별 예측값 출력
        x = self.fc(x)
        return x

# Custom ResNet-18 모델 생성 (각 레이어의 블록 수는 2개씩)
model = CustomResNet(Block, [2, 2, 2, 2], num_classes=10)
